In [1]:
import pandas as pd 
import os 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
!ls week1/

ls: week1/: No such file or directory


### Question 1:

#### Read the data for January. How many columns are there? 
#### Answer: 19

In [3]:
df_jan= pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_jan['month']= 'jan'
print(f'jan data shape: {df_jan.shape}')
df_feb= pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_feb['month']= 'feb'
print(f'feb data shape: {df_feb.shape}')

df_merge= pd.concat([df_jan, df_feb], ignore_index=True)
print('merged data frame size is: ', df_merge.shape)
df_merge.head()

jan data shape: (3066766, 20)
feb data shape: (2913955, 20)
merged data frame size is:  (5980721, 21)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,month,Airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,jan,NaN
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,jan,NaN
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,jan,NaN
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,jan,NaN
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,jan,NaN


### Question 2:

#### What's the standard deviation of the trips duration in January?
#### 42.59


In [4]:
df_jan['duration']= df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration']= df_jan['duration'].dt.seconds/60
df_jan['duration'].describe()

count    3.066766e+06
mean     1.565303e+01
std      4.162919e+01
min      0.000000e+00
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.439800e+03
Name: duration, dtype: float64

### Question 3: 

#### What fraction of the records left after you dropped the outliers?
#### Answer: 98%


In [5]:
df_merge['duration']= df_merge['tpep_dropoff_datetime'] - df_merge['tpep_pickup_datetime']
df_merge['duration']= df_merge['duration'].dt.seconds/60
df_filtered= df_merge[(df_merge['duration']>=1) & (df_merge['duration']<=60)]
print('ratio of trip between 1 and 60 mins: ', len(df_filtered)/len(df_merge))

ratio of trip between 1 and 60 mins:  0.9806722299869866


### Question 4: 

#### What's the dimensionality of this matrix (number of columns)?

#### Answer: 515

In [6]:
# df_filtered.head()
categorical= ['PULocationID', 'DOLocationID']
numerical= ['duration']

df_model= df_filtered[['PULocationID', 'DOLocationID', 'duration', 'month']]
df_model['PULocationID']= df_model['PULocationID'].astype(str)
df_model['DOLocationID']= df_model['DOLocationID'].astype(str)
print(df_model.dtypes)


df_train= df_model[df_model.month=='jan'][categorical+numerical]
train_dicts= df_train[categorical].to_dict(orient='records')
y_train=  df_train[numerical].values


df_val= df_model[df_model.month=='feb'][categorical+numerical]
val_dicts= df_val[categorical].to_dict(orient='records')
y_val= df_val[numerical]


/var/folders/1t/6xqd_4t12zs1c072dm6yn3gm0000gn/T/ipykernel_70683/1476452217.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['PULocationID']= df_model['PULocationID'].astype(str)
/var/folders/1t/6xqd_4t12zs1c072dm6yn3gm0000gn/T/ipykernel_70683/1476452217.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['DOLocationID']= df_model['DOLocationID'].astype(str)


PULocationID     object
DOLocationID     object
duration        float64
month            object
dtype: object


In [11]:
dv= DictVectorizer()
x_train= dv.fit_transform(train_dicts)
x_val= dv.transform(val_dicts)
len(dv.feature_names_) #no of feature +1 this 1 value is the duration column which isn't a categorical feature

515

### Question 5:

#### What's the RMSE on train?
#### Answer: 7.64

In [16]:
lr= LinearRegression()
lr.fit(x_train, df_train.duration.values)
y_pred= lr.predict(x_train)

mean_squared_error(df_train.duration.values, y_pred, squared=False)

7.6492610905590945

### Question 6:

#### What's the RMSE on validation?
#### Answer: 7.81

In [10]:
y_val_pred= lr.predict(x_val)
mean_squared_error(df_val.duration.values, y_val_pred, squared=False)


7.81182099585697